In [1]:
import mido
import os
import tqdm
from structures import *

In [2]:
def get_duration(mid, time):
    return round(time/(mid.ticks_per_beat/(128/4)))
    
def process_file(name, outf):
    try: #for i in range(0, 1):
        try:
            mid = mido.MidiFile(name)
        except Exception as e:
            print('Broken midi: ', e)
            return
        
        song = Song()
        song.name = name
        try:
            song.bpm = int(mido.tempo2bpm(list(filter(lambda msg: msg.type == 'set_tempo', mid))[0].tempo))
        except Exception as e:
            song.bpm = 196
        
        for mid_track in mid.tracks:
            try:
                # Треки с метаданными не обрабатываем
                if len(list(filter(lambda msg: msg.type == 'note_on', mid_track))) == 0:
                    continue
                    
                track = Track()
                
                try:
                    track.track_name = list(filter(lambda msg: msg.type == 'track_name', mid_track))[0].name
                except Exception:
                    track.track_name = ''
                    
                try:
                    track.instrument_name = list(filter(lambda msg: msg.type == 'instrument_name', mid_track))[0].name
                except Exception:
                    track.instrument_name = ''
                    
                try:
                    track.program = list(filter(lambda msg: msg.type == 'program_change', mid_track))[0].program
                except Exception:
                    track.program = -1
                #if program_change >= 97 or (33 <= program_change <= 40):
                #    continue
                    
                try:
                    key_signature = list(filter(lambda msg: msg.type == 'key_signature', mid_track))[0].key
                    song.key_signature = key_signature
                except Exception:
                    key_signature = ''
                
                notes_on = list(filter(lambda msg: msg.type == 'note_on', mid_track))
                chords = [Chord([], 0, 0)]
                for i, note_on in enumerate(notes_on):
                    cur_duration = 0
                    
                    if int(note_on.velocity) != 0: # Если это note_on
                        if note_on.time != 0:
                            chords += [ Chord([], get_duration(mid, note_on.time), 0) ] # Пауза
                        
                        for j in range(i+1, len(notes_on)):
                            cur_duration += get_duration(mid, notes_on[j].time)
                            if notes_on[j].note == note_on.note and notes_on[j].velocity == 0: # Нашли конец
                                if note_on.time == 0: # Если это продолжение аккорда
                                    chords[-1].notes += [Note(note_on.note)]
                                    #if chords[-1].length != cur_duration or chords[-1].beat_force != note_on.velocity:
                                    #    print("No")
                                    chords[-1].duration = cur_duration
                                    chords[-1].velocity = note_on.velocity
                                else:
                                    chords += [Chord([], cur_duration, note_on.velocity)]
                                    chords[-1].notes += [Note(note_on.note)]
                                break
                    
                track.chords = chords
                
                song.add_track(track)
            except Exception as e:
                print('Track broken', e)
                break
        song.dump(outf)
        return song
    except Exception as e:
        print('MIDI broken %s'%name)
        print(e)


In [3]:
outf = open('test.txt', 'wb')
song = process_file('BackInBlack.mid', outf)
outf.close()
s = Song()
s.load('test.txt')

In [4]:
print(song)
print(s)

'BackInBlack.mid' 8 94
'Guitar 1' 'Guitar 1' 29
'Guitar 1  cont.' 'Guitar 1  cont.' 29
'Guitar 2' 'Guitar 2' 30
'Guitar 2  cont.' 'Guitar 2  cont.' 30
'Bass' 'Bass' 33
'Snare' 'Snare' 114
'Bass Drum' 'Bass Drum' 112
'Cymbal' 'Cymbal' 118

'BackInBlack.mid' 8 94
'Guitar 1' 'Guitar 1' 29
'Guitar 1  cont.' 'Guitar 1  cont.' 29
'Guitar 2' 'Guitar 2' 30
'Guitar 2  cont.' 'Guitar 2  cont.' 30
'Bass' 'Bass' 33
'Snare' 'Snare' 114
'Bass Drum' 'Bass Drum' 112
'Cymbal' 'Cymbal' 118



In [9]:
def recursive_walk_directory(dirname):
    outf = open('out.txt', 'wb')
    pb = tqdm.tqdm_notebook(total=128)
    for root, directories, filenames in os.walk(dirname):
        for filename in filenames:
            if filename[-4:].lower() == '.mid':
                process_file(os.path.join(root, filename), outf)
                pb.update(n=1)
            else:
                print(filename)
    outf.close()

In [10]:
recursive_walk_directory('metallica/')

HBox(children=(IntProgress(value=0, max=128), HTML(value='')))

Broken midi:  data byte must be in range 0..127


In [13]:
inf = open('out.txt', 'rb')
pb = tqdm.tqdm_notebook(total=128)
songs = []
while True:
    try:
        song = Song()
        song.undump(inf)
        songs.append(song)
        pb.update(n=1)
    except Exception as e:
        print(e)
        break
inf.close()

HBox(children=(IntProgress(value=0, max=128), HTML(value='')))


Ran out of input


In [14]:
len(songs)

127